In [2]:
from __future__ import division

import argparse, time, logging, os, sys, math

import numpy as np
import pandas as pd
import mxnet as mx
import gluoncv as gcv
from mxnet import gluon, nd, init, context
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms

from gluoncv.data.transforms import video
from gluoncv.data import VideoClsCustom
from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, LRSequential, LRScheduler, split_and_load, TrainingHistory

In [26]:
metadata = pd.read_csv("metadata.csv")
metadata.shape

(21083, 12)

In [103]:
actions = np.array(['hello', 'love', 'thank you'])
train_txt = metadata[(metadata['gloss'].isin(actions)) & (metadata['split'] == 'train')][['video_id', 'fps', 'gloss']]#.unique()
#train_txt['video_id'] = train_txt['video_id'].astype(str) + '.mp4'
label_map = {label:num for num, label in enumerate(actions)}
train_txt = train_txt.replace({"gloss": label_map})
train_txt

,video_id,fps,gloss
5579,57653,25,2
5580,69502,25,2
5581,68796,25,2
5583,57664,25,2
5584,66598,25,2
5585,57655,25,2
5586,57665,25,2
5587,57666,25,2
5591,57658,25,2
5592,57659,25,2


In [115]:
train_txt.to_csv(r'train.txt', header=None, index=None, sep=' ', mode='a')

In [113]:
train_txt.shape

(27, 3)

In [116]:
num_gpus = 0
ctx = [mx.gpu(i) for i in range(num_gpus)]
transform_train = video.VideoGroupTrainTransform(size=(224, 224), scale_ratios=[1.0, 0.8], mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
per_device_batch_size = 5
num_workers = 0
batch_size = per_device_batch_size * num_gpus

train_dataset = VideoClsCustom(root=os.path.expanduser('~\\Sign-Language-Translator\\videos'),
                               setting=os.path.expanduser('~\\Sign-Language-Translator\\notebooks\\train.txt'),
                               train=True,
                               #new_length=32,
                               transform=transform_train)
print('Load %d training samples.' % len(train_dataset))
train_data = gluon.data.DataLoader(train_dataset, batch_size=batch_size,
                                   shuffle=True, num_workers=num_workers)

Load 27 training samples.


In [117]:
net = get_model(name='i3d_resnet50_v1_custom', nclass=3)
net.collect_params().reset_ctx(ctx)
print(net)

conv24_weight is done with shape:  (64, 3, 5, 7, 7)
batchnorm24_gamma is done with shape:  (64,)
batchnorm24_beta is done with shape:  (64,)
batchnorm24_running_mean is done with shape:  (64,)
batchnorm24_running_var is done with shape:  (64,)
layer1_0_conv0_weight is done with shape:  (64, 64, 3, 1, 1)
layer1_0_batchnorm0_gamma is done with shape:  (64,)
layer1_0_batchnorm0_beta is done with shape:  (64,)
layer1_0_batchnorm0_running_mean is done with shape:  (64,)
layer1_0_batchnorm0_running_var is done with shape:  (64,)
layer1_0_conv1_weight is done with shape:  (64, 64, 1, 3, 3)
layer1_0_batchnorm1_gamma is done with shape:  (64,)
layer1_0_batchnorm1_beta is done with shape:  (64,)
layer1_0_batchnorm1_running_mean is done with shape:  (64,)
layer1_0_batchnorm1_running_var is done with shape:  (64,)
layer1_0_conv2_weight is done with shape:  (256, 64, 1, 1, 1)
layer1_0_batchnorm2_gamma is done with shape:  (256,)
layer1_0_batchnorm2_beta is done with shape:  (256,)
layer1_0_batchnor

In [118]:
# Learning rate decay factor
lr_decay = 0.1
# Epochs where learning rate decays
lr_decay_epoch = [40, 80, 100]

# Stochastic gradient descent
optimizer = 'sgd'
# Set parameters
optimizer_params = {'learning_rate': 0.001, 'wd': 0.0001, 'momentum': 0.9}

# Define our trainer for net
trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)

In [119]:
loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()

In [120]:
train_metric = mx.metric.Accuracy()
train_history = TrainingHistory(['training-acc'])

In [121]:
for i, batch in enumerate(train_data):
    print(i, batch)

RuntimeError: Could not load file C:\Users\rbcor\Sign-Language-Translator\videos\34128\img_00008.jpg starting at frame 8. Check data path.

In [102]:
epochs = 3
lr_decay_count = 0

for epoch in range(epochs):
    tic = time.time()
    train_metric.reset()
    train_loss = 0

    # Learning rate decay
    if epoch == lr_decay_epoch[lr_decay_count]:
        trainer.set_learning_rate(trainer.learning_rate*lr_decay)
        lr_decay_count += 1

    # Loop through each batch of training data
    for i, batch in enumerate(train_data):
        # Extract data and label
        data = split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        label = split_and_load(batch[1], ctx_list=ctx, batch_axis=0)

        # AutoGrad
        with ag.record():
            output = []
            for _, X in enumerate(data):
                X = X.reshape((-1,) + X.shape[2:])
                pred = net(X)
                output.append(pred)
            loss = [loss_fn(yhat, y) for yhat, y in zip(output, label)]

        # Backpropagation
        for l in loss:
            l.backward()

        # Optimize
        trainer.step(batch_size)

        # Update metrics
        train_loss += sum([l.mean().asscalar() for l in loss])
        train_metric.update(label, output)

        if i == 100:
            break

    name, acc = train_metric.get()

    # Update history and print metrics
    train_history.update([acc])
    print('[Epoch %d] train=%f loss=%f time: %f' %
        (epoch, acc, train_loss / (i+1), time.time()-tic))

# We can plot the metric scores with:
train_history.plot()

RuntimeError: Could not load file C:\Users\rbcor\Sign-Language-Translator\videos\66598.mp4\img_00011.jpg starting at frame 11. Check data path.